In [37]:
import csv
import pandas as pd
import numpy as np
import math
import scipy
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [38]:
def dateToIndex(date):
    datearr = date.split("/")
    month = int(datearr[0])
    year = int(datearr[2])
    return (year-2010)*12+month-1

In [39]:
appVolDat = pd.read_csv('filingVolumes.csv')
prediction = appVolDat.iloc[1405]
appVolDat = appVolDat.iloc[1320:1404]
appVolDat = appVolDat.reset_index(drop=True)
maximum = max(appVolDat.volume)
appVolDat['vol'] = appVolDat.volume/maximum
appVolDat['vol'] = np.log(appVolDat.vol)
print(appVolDat)

        month  volume       vol
0    1/1/2010   31365 -0.974123
1    2/1/2010   33237 -0.916152
2    3/1/2010   43055 -0.657338
3    4/1/2010   38781 -0.761886
4    5/1/2010   36019 -0.835769
5    6/1/2010   42086 -0.680101
6    7/1/2010   36667 -0.817939
7    8/1/2010   37406 -0.797985
8    9/1/2010   40013 -0.730612
9   10/1/2010   38245 -0.775803
10  11/1/2010   38164 -0.777923
11  12/1/2010   45032 -0.612443
12   1/1/2011   35259 -0.857095
13   2/1/2011   35057 -0.862841
14   3/1/2011   45603 -0.599843
15   4/1/2011   38163 -0.777950
16   5/1/2011   38934 -0.757948
17   6/1/2011   43581 -0.645195
18   7/1/2011   35761 -0.842958
19   8/1/2011   41170 -0.702106
20   9/1/2011   47684 -0.555220
21  10/1/2011   37863 -0.785842
22  11/1/2011   39011 -0.755972
23  12/1/2011   46466 -0.581095
24   1/1/2012   36431 -0.824396
25   2/1/2012   38898 -0.758873
26   3/1/2012   46164 -0.587616
27   4/1/2012   40774 -0.711771
28   5/1/2012   42497 -0.670383
29   6/1/2012   45306 -0.606377
..      

In [40]:
for i in range(1,13):
    appVolDat['volShift'+str(i)] = appVolDat.vol.shift(i)

In [41]:
appVolDat

,month,volume,vol,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12
0,1/1/2010,31365,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/1/2010,33237,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3/1/2010,43055,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/1/2010,38781,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5/1/2010,36019,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6/1/2010,42086,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7/1/2010,36667,-0.817939,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN
7,8/1/2010,37406,-0.797985,-0.817939,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN
8,9/1/2010,40013,-0.730612,-0.797985,-0.817939,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN
9,10/1/2010,38245,-0.775803,-0.730612,-0.797985,-0.817939,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN


In [42]:
start = dateToIndex('1/1/2011')
end = len(appVolDat)
log_model = LinearRegression().fit( 
    
    np.c_[appVolDat.volShift1[start:end], 
          appVolDat.volShift2[start:end], 
          appVolDat.volShift3[start:end], 
          appVolDat.volShift4[start:end], 
          appVolDat.volShift5[start:end], 
          appVolDat.volShift6[start:end], 
          appVolDat.volShift7[start:end], 
          appVolDat.volShift8[start:end], 
          appVolDat.volShift9[start:end], 
          appVolDat.volShift10[start:end],
          appVolDat.volShift11[start:end],
          appVolDat.volShift12[start:end]], 
    appVolDat.vol[start:end])

In [43]:
log_model.coef_

array([ 0.32048627,  0.27330313,  0.33834874,  0.03834052,  0.13087862,
        0.10807778,  0.04398031, -0.18274453,  0.13257573, -0.14998679,
        0.0255287 ,  0.45041368])

In [44]:
appVolDat['predict'] = np.zeros_like(appVolDat.volume)
for i in range(start, len(appVolDat)):
    xhat = math.pow(log_model.predict(

        np.c_[appVolDat.volShift1[i], 
              appVolDat.volShift2[i], 
              appVolDat.volShift3[i], 
              appVolDat.volShift4[i], 
              appVolDat.volShift5[i], 
              appVolDat.volShift6[i], 
              appVolDat.volShift7[i], 
              appVolDat.volShift8[i], 
              appVolDat.volShift9[i], 
              appVolDat.volShift10[i],
              appVolDat.volShift11[i],
              appVolDat.volShift12[i]]

    )[0], 10)*maximum
    appVolDat.predict.iloc[i] = int(round(xhat))
    
appVolDat

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,month,volume,vol,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12,predict
0,1/1/2010,31365,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2/1/2010,33237,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3/1/2010,43055,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4/1/2010,38781,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5/1/2010,36019,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,6/1/2010,42086,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,7/1/2010,36667,-0.817939,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,NaN,0
7,8/1/2010,37406,-0.797985,-0.817939,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,NaN,0
8,9/1/2010,40013,-0.730612,-0.797985,-0.817939,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,NaN,0
9,10/1/2010,38245,-0.775803,-0.730612,-0.797985,-0.817939,-0.680101,-0.835769,-0.761886,-0.657338,-0.916152,-0.974123,NaN,NaN,NaN,0


In [45]:
pd.read_csv('costOfPatent.csv').iloc[72:156]
#appVolDat['trend1'] = list(pd.read_csv('costOfPatent.csv').iloc[73:].trends)

,month,trends
72,1/1/2010,52
73,2/1/2010,36
74,3/1/2010,57
75,4/1/2010,30
76,5/1/2010,44
77,6/1/2010,59
78,7/1/2010,31
79,8/1/2010,33
80,9/1/2010,22
81,10/1/2010,21
